In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from cryptobot.yahoo_market_data import get_yahoo_data
from cryptobot.data_engineering import *
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn import set_config; set_config(display='diagram')
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.metrics import MAPE
from sklearn.preprocessing import OrdinalEncoder


2022-06-09 16:16:39.256351: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-09 16:16:39.256492: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
"""
Get BASE ETHEREUM data per hour from Binance (downloaded .csv file)
"""

BASE_DATA_PATH = "data/candles-ETHUSDT.csv"
COLUMN_NAMES = ["open_time","open","high","low", "close",
                         "volume", "close_time" ,"quote_asset_volume",
                         "number_of_trades", "taker_buy_base_asset_volume",
                         "taker_buy_quote_asset_volume", "ignore"]

def get_data_without_headers(path):
    df = pd.read_csv(path,
                 header="infer")
    df.open_time = df.open_time.apply(lambda x: datetime.utcfromtimestamp(x/1000))
    df.close_time = df.close_time.apply(lambda x: datetime.utcfromtimestamp(x/1000))
    df["close_time_min"] = df["close_time"].apply(lambda x: x.strftime("%Y-%m-%d-%H-%M"))
    df["close_time_day"] = df["close_time"].apply(lambda x: x.strftime("%Y-%m-%d"))
    
    return df

df = get_data_without_headers(BASE_DATA_PATH)
df

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,close_time_min,close_time_day
0,2018-02-01 03:00:00,1150.51,1164.99,1125.01,1140.00,7622.31840,2018-02-01 03:59:59.999,8.731269e+06,12304,4249.14779,4.870025e+06,2018-02-01-03-59,2018-02-01
1,2018-02-01 04:00:00,1140.00,1154.72,1130.18,1152.51,3899.48396,2018-02-01 04:59:59.999,4.461803e+06,8404,1925.63184,2.203351e+06,2018-02-01-04-59,2018-02-01
2,2018-02-01 05:00:00,1152.51,1164.42,1146.50,1159.13,3462.28586,2018-02-01 05:59:59.999,4.005314e+06,7597,1814.24364,2.098444e+06,2018-02-01-05-59,2018-02-01
3,2018-02-01 06:00:00,1159.02,1159.02,1140.01,1142.99,3400.69303,2018-02-01 06:59:59.999,3.907627e+06,9472,1637.92439,1.881936e+06,2018-02-01-06-59,2018-02-01
4,2018-02-01 07:00:00,1142.99,1151.00,1135.00,1137.10,2589.27888,2018-02-01 07:59:59.999,2.962427e+06,6319,1601.73117,1.833291e+06,2018-02-01-07-59,2018-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37798,2022-05-30 23:00:00,1999.41,2013.00,1984.03,1998.78,50617.55410,2022-05-30 23:59:59.999,1.012467e+08,43653,26245.35190,5.249899e+07,2022-05-30-23-59,2022-05-30
37799,2022-05-31 00:00:00,1998.78,2016.45,1980.01,1992.86,44752.71670,2022-05-31 00:59:59.999,8.935535e+07,44172,21081.28480,4.209205e+07,2022-05-31-00-59,2022-05-31
37800,2022-05-31 01:00:00,1992.85,1993.48,1976.55,1986.59,21968.64670,2022-05-31 01:59:59.999,4.363847e+07,22420,9932.26980,1.973056e+07,2022-05-31-01-59,2022-05-31
37801,2022-05-31 02:00:00,1986.59,1999.90,1981.71,1996.26,18712.98890,2022-05-31 02:59:59.999,3.728188e+07,22148,9915.71620,1.975392e+07,2022-05-31-02-59,2022-05-31


In [3]:
"""
Get FEAR AND GREED data from Binance (downloaded .csv file)
"""

FEAR_GREED_PATH = "data/fear_greed_index.csv"

def get_data_fg_with_headers(path):
    fg = pd.read_csv(path)
    fg["timestamp"] = fg.timestamp.apply(lambda x: datetime.utcfromtimestamp(int(x)))
    fg["close_time_day"] = fg["timestamp"].apply(lambda x: x.strftime("%Y-%m-%d"))
    fg.drop(columns=["Unnamed: 0","timestamp"],inplace=True)
    return fg

fg = get_data_fg_with_headers(FEAR_GREED_PATH)
fg.head(3)

,value,value_classification,close_time_day
0,13,Extreme Fear,2022-06-06
1,10,Extreme Fear,2022-06-05
2,14,Extreme Fear,2022-06-04


In [4]:
"""
Get BITCOIN DOMINANCE data from Binance (downloaded .csv file)
"""

BTC_DOM_PATH = "data/BTC Dominance - Trading View - 1day.csv"

def get_data_btc_with_headers(path):
    btc_dom = pd.read_csv(path)
    btc_dom["time"] = btc_dom.time.apply(lambda x: datetime.utcfromtimestamp(int(x)))
    btc_dom["close_time_day"] = btc_dom["time"].apply(lambda x: x.strftime("%Y-%m-%d"))
    btc_dom["btc_avg"]= btc_dom["open"]+btc_dom["close"]+btc_dom["high"]+btc_dom["low"]/4
    btc_dom.drop(columns=["time","open","high","low","close","Volume","Volume MA"], inplace=True)

    return btc_dom

btc_dom = get_data_btc_with_headers(BTC_DOM_PATH)
btc_dom


,close_time_day,btc_avg
0,2014-04-01,323.016114
1,2014-04-02,322.943635
2,2014-04-03,323.063574
3,2014-04-04,323.113947
4,2014-04-05,323.200215
...,...,...
2981,2022-06-05,152.360945
2982,2022-06-06,153.657384
2983,2022-06-07,154.868802
2984,2022-06-08,154.631329


In [5]:
"""
Merging BASE ETHEREUM data from Binance ===> with FEAR AND GREED data
"""

def merge_df_fg(df,fg):
    df = df.merge(fg, on = "close_time_day", how="left")
    return df

df_fg =  merge_df_fg(df,fg)
df_fg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37803 entries, 0 to 37802
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     37803 non-null  datetime64[ns]
 1   open                          37803 non-null  float64       
 2   high                          37803 non-null  float64       
 3   low                           37803 non-null  float64       
 4   close                         37803 non-null  float64       
 5   volume                        37803 non-null  float64       
 6   close_time                    37803 non-null  datetime64[ns]
 7   quote_asset_volume            37803 non-null  float64       
 8   number_of_trades              37803 non-null  int64         
 9   taker_buy_base_asset_volume   37803 non-null  float64       
 10  taker_buy_quote_asset_volume  37803 non-null  float64       
 11  close_time_min              

In [6]:
"""
Merging BASE ETHEREUM data from Binance + FEAR AND GREED data ===> with BITCOIN DOMINANCE data
"""

def merge_df_btc(df,btc):
    df = df.merge(btc, on = "close_time_day", how="left" )
    return df
                  
df_fg =  merge_df_btc(df_fg,btc_dom)
df_fg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37803 entries, 0 to 37802
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     37803 non-null  datetime64[ns]
 1   open                          37803 non-null  float64       
 2   high                          37803 non-null  float64       
 3   low                           37803 non-null  float64       
 4   close                         37803 non-null  float64       
 5   volume                        37803 non-null  float64       
 6   close_time                    37803 non-null  datetime64[ns]
 7   quote_asset_volume            37803 non-null  float64       
 8   number_of_trades              37803 non-null  int64         
 9   taker_buy_base_asset_volume   37803 non-null  float64       
 10  taker_buy_quote_asset_volume  37803 non-null  float64       
 11  close_time_min              

In [8]:
"""
Get and Merge PERCENT SUPPLY IN PROFIT data from Glassnode (downloaded .json file)
"""

JSON_PATH = "data/percent-supply-in-profit-eth-24h.json"

def merge_perc_supp(df, path):
    data = pd.read_json(path)
    data["close_time_day"]=data["t"].apply(lambda x: x[0:10])
    data.drop(columns="t",inplace=True)
    data = df.merge(data, on = "close_time_day", how="left")
    return data

df_fg = merge_perc_supp(df_fg,JSON_PATH)
df_fg

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,close_time_min,close_time_day,value,value_classification,btc_avg,v
0,2018-02-01 03:00:00,1150.51,1164.99,1125.01,1140.00,7622.31840,2018-02-01 03:59:59.999,8.731269e+06,12304,4249.14779,4.870025e+06,2018-02-01-03-59,2018-02-01,30.0,Fear,123.645560,0.875434
1,2018-02-01 04:00:00,1140.00,1154.72,1130.18,1152.51,3899.48396,2018-02-01 04:59:59.999,4.461803e+06,8404,1925.63184,2.203351e+06,2018-02-01-04-59,2018-02-01,30.0,Fear,123.645560,0.875434
2,2018-02-01 05:00:00,1152.51,1164.42,1146.50,1159.13,3462.28586,2018-02-01 05:59:59.999,4.005314e+06,7597,1814.24364,2.098444e+06,2018-02-01-05-59,2018-02-01,30.0,Fear,123.645560,0.875434
3,2018-02-01 06:00:00,1159.02,1159.02,1140.01,1142.99,3400.69303,2018-02-01 06:59:59.999,3.907627e+06,9472,1637.92439,1.881936e+06,2018-02-01-06-59,2018-02-01,30.0,Fear,123.645560,0.875434
4,2018-02-01 07:00:00,1142.99,1151.00,1135.00,1137.10,2589.27888,2018-02-01 07:59:59.999,2.962427e+06,6319,1601.73117,1.833291e+06,2018-02-01-07-59,2018-02-01,30.0,Fear,123.645560,0.875434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37798,2022-05-30 23:00:00,1999.41,2013.00,1984.03,1998.78,50617.55410,2022-05-30 23:59:59.999,1.012467e+08,43653,26245.35190,5.249899e+07,2022-05-30-23-59,2022-05-30,10.0,Extreme Fear,151.034615,NaN
37799,2022-05-31 00:00:00,1998.78,2016.45,1980.01,1992.86,44752.71670,2022-05-31 00:59:59.999,8.935535e+07,44172,21081.28480,4.209205e+07,2022-05-31-00-59,2022-05-31,16.0,Extreme Fear,151.648366,NaN
37800,2022-05-31 01:00:00,1992.85,1993.48,1976.55,1986.59,21968.64670,2022-05-31 01:59:59.999,4.363847e+07,22420,9932.26980,1.973056e+07,2022-05-31-01-59,2022-05-31,16.0,Extreme Fear,151.648366,NaN
37801,2022-05-31 02:00:00,1986.59,1999.90,1981.71,1996.26,18712.98890,2022-05-31 02:59:59.999,3.728188e+07,22148,9915.71620,1.975392e+07,2022-05-31-02-59,2022-05-31,16.0,Extreme Fear,151.648366,NaN


In [9]:
""" 
Merging ===> with YAHOO FINANCIAL INDICES data with get_yahoo_data() fuction that feeds from Yahoo API

Loop for importing datasets with various financial indices   
    
    '^IXIC' - Nasdaq compound index
    '^GSPC' - S&P 500 index
    'GC=F' - Gold
    '^DJI' - Dow Jones Industrial Average
    '^TNX' = S&P 500 US T-bills (10 year)
    'DX-Y.NYB'= US Dollar/USDX - Index - Cash

"""

# Index '^TNX' with "Volume" 0 value in all rows

INDEX_LIST = ['^IXIC','^GSPC','GC=F','^DJI', '^TNX','DX-Y.NYB']
START_DATE = "2020-05-05"
END_DATE = "2022-06-07"

for i in INDEX_LIST:
    yd = get_yahoo_data(i, START_DATE, END_DATE)
    yd[f'{i}_Date'] = yd[f'{i}_Date'].apply(lambda x: x.strftime("%Y-%m-%d"))
    yd.rename(columns = {f'{i}_Date':"close_time_day"}, inplace = True)
    yd[f'{i}_avg']= yd[f'{i}_Open']+yd[f'{i}_Close']+yd[f'{i}_High']+yd[f'{i}_Low']/4
    df_fg = df_fg.merge(yd, on = "close_time_day", how="left" )
    df_fg.drop(columns=[f'{i}_timestamp',f'{i}_Open',f'{i}_Close',f'{i}_High',f'{i}_Low'], inplace=True)
df_fg


,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,...,value,value_classification,btc_avg,v,^IXIC_avg,^GSPC_avg,GC=F_avg,^DJI_avg,^TNX_avg,DX-Y.NYB_avg
0,2018-02-01 03:00:00,1150.51,1164.99,1125.01,1140.00,7622.31840,2018-02-01 03:59:59.999,8.731269e+06,12304,4249.14779,...,30.0,Fear,123.645560,0.875434,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-02-01 04:00:00,1140.00,1154.72,1130.18,1152.51,3899.48396,2018-02-01 04:59:59.999,4.461803e+06,8404,1925.63184,...,30.0,Fear,123.645560,0.875434,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-02-01 05:00:00,1152.51,1164.42,1146.50,1159.13,3462.28586,2018-02-01 05:59:59.999,4.005314e+06,7597,1814.24364,...,30.0,Fear,123.645560,0.875434,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-02-01 06:00:00,1159.02,1159.02,1140.01,1142.99,3400.69303,2018-02-01 06:59:59.999,3.907627e+06,9472,1637.92439,...,30.0,Fear,123.645560,0.875434,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-02-01 07:00:00,1142.99,1151.00,1135.00,1137.10,2589.27888,2018-02-01 07:59:59.999,2.962427e+06,6319,1601.73117,...,30.0,Fear,123.645560,0.875434,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37798,2022-05-30 23:00:00,1999.41,2013.00,1984.03,1998.78,50617.55410,2022-05-30 23:59:59.999,1.012467e+08,43653,26245.35190,...,10.0,Extreme Fear,151.034615,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37799,2022-05-31 00:00:00,1998.78,2016.45,1980.01,1992.86,44752.71670,2022-05-31 00:59:59.999,8.935535e+07,44172,21081.28480,...,16.0,Extreme Fear,151.648366,NaN,39394.634766,13477.799561,6013.299988,107579.014648,9.258,330.712502
37800,2022-05-31 01:00:00,1992.85,1993.48,1976.55,1986.59,21968.64670,2022-05-31 01:59:59.999,4.363847e+07,22420,9932.26980,...,16.0,Extreme Fear,151.648366,NaN,39394.634766,13477.799561,6013.299988,107579.014648,9.258,330.712502
37801,2022-05-31 02:00:00,1986.59,1999.90,1981.71,1996.26,18712.98890,2022-05-31 02:59:59.999,3.728188e+07,22148,9915.71620,...,16.0,Extreme Fear,151.648366,NaN,39394.634766,13477.799561,6013.299988,107579.014648,9.258,330.712502


In [10]:
"""
Creating new features using data_engineering.py defined fuctions
"""

df_fg = ema_metric(df_fg,df_fg["close"],10)
df_fg = ema_metric(df_fg,df_fg["close"],50)
df_fg = ema_metric(df_fg,df_fg["close"],200)
df_fg = rsi_metric(df_fg,df_fg["close"],14)
df_fg = adx_dmp_dmn_metric(df_fg,df_fg["high"],df_fg["low"],df_fg["close"],14)
df_fg = atr_metric(df_fg,df_fg["high"],df_fg["low"],df_fg["close"],14)
df_fg


,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,...,^TNX_avg,DX-Y.NYB_avg,EMA_10,EMA_50,EMA_200,RSI_14,ADX_14,DMP_14,DMN_14,ATR_14
0,2018-02-01 03:00:00,1150.51,1164.99,1125.01,1140.00,7622.31840,2018-02-01 03:59:59.999,8.731269e+06,12304,4249.14779,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-02-01 04:00:00,1140.00,1154.72,1130.18,1152.51,3899.48396,2018-02-01 04:59:59.999,4.461803e+06,8404,1925.63184,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-02-01 05:00:00,1152.51,1164.42,1146.50,1159.13,3462.28586,2018-02-01 05:59:59.999,4.005314e+06,7597,1814.24364,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-02-01 06:00:00,1159.02,1159.02,1140.01,1142.99,3400.69303,2018-02-01 06:59:59.999,3.907627e+06,9472,1637.92439,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-02-01 07:00:00,1142.99,1151.00,1135.00,1137.10,2589.27888,2018-02-01 07:59:59.999,2.962427e+06,6319,1601.73117,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37798,2022-05-30 23:00:00,1999.41,2013.00,1984.03,1998.78,50617.55410,2022-05-30 23:59:59.999,1.012467e+08,43653,26245.35190,...,NaN,NaN,1953.190456,1871.405020,1885.043501,81.600631,49.306548,38.204494,6.974402,24.620486
37799,2022-05-31 00:00:00,1998.78,2016.45,1980.01,1992.86,44752.71670,2022-05-31 00:59:59.999,8.935535e+07,44172,21081.28480,...,9.258,330.712502,1960.403100,1876.167961,1886.116302,78.448296,50.395425,34.299453,7.389129,25.464737
37800,2022-05-31 01:00:00,1992.85,1993.48,1976.55,1986.59,21968.64670,2022-05-31 01:59:59.999,4.363847e+07,22420,9932.26980,...,9.258,330.712502,1965.164355,1880.498237,1887.116041,75.137547,51.119054,32.630668,8.023957,24.855113
37801,2022-05-31 02:00:00,1986.59,1999.90,1981.71,1996.26,18712.98890,2022-05-31 02:59:59.999,3.728188e+07,22148,9915.71620,...,9.258,330.712502,1970.818108,1885.037914,1888.202050,76.766123,51.922373,32.772617,7.596319,24.379033


In [11]:
"""
Filling NAN values:

df.fillna(method='ffill') 
       
    This method propagate[s] last valid observation forward to next valid
        
df.fillna(method="bfill")
     
     This method propagate[s] last valid observation backward to last valid
"""

df_fg=df_fg.fillna(method="bfill").fillna(method="ffill")
df_fg.isna().sum()

open_time                       0
open                            0
high                            0
low                             0
close                           0
volume                          0
close_time                      0
quote_asset_volume              0
number_of_trades                0
taker_buy_base_asset_volume     0
taker_buy_quote_asset_volume    0
close_time_min                  0
close_time_day                  0
value                           0
value_classification            0
btc_avg                         0
v                               0
^IXIC_avg                       0
^GSPC_avg                       0
GC=F_avg                        0
^DJI_avg                        0
^TNX_avg                        0
DX-Y.NYB_avg                    0
EMA_10                          0
EMA_50                          0
EMA_200                         0
RSI_14                          0
ADX_14                          0
DMP_14                          0
DMN_14        

In [12]:
"""
Creating Target Feature  ===> Difference between open ETH value and close ETH value
""" 

def define_target(df):
    df["target"] = (df.open - df.close).apply(lambda x: 0 if x <=0 else 1)
    return df

df = define_target(df_fg)
df_fg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37803 entries, 0 to 37802
Data columns (total 32 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     37803 non-null  datetime64[ns]
 1   open                          37803 non-null  float64       
 2   high                          37803 non-null  float64       
 3   low                           37803 non-null  float64       
 4   close                         37803 non-null  float64       
 5   volume                        37803 non-null  float64       
 6   close_time                    37803 non-null  datetime64[ns]
 7   quote_asset_volume            37803 non-null  float64       
 8   number_of_trades              37803 non-null  int64         
 9   taker_buy_base_asset_volume   37803 non-null  float64       
 10  taker_buy_quote_asset_volume  37803 non-null  float64       
 11  close_time_min              

In [13]:
"""
Some Minor changes before preprocessing:
     2 columns from timestamp to int
     Dropping columns created to merge
"""

df_fg["open_time"]= pd.to_datetime(df_fg["open_time"]).astype(np.int64)
df_fg["close_time"] = pd.to_datetime(df_fg["close_time"]).astype(np.int64)
df_fg.drop(columns=["close_time_day","close_time_min"], inplace=True)
df_fg.head()

/tmp/ipykernel_15946/2988358372.py:7: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  df_fg["open_time"]= pd.to_datetime(df_fg["open_time"]).astype(np.int64)
/tmp/ipykernel_15946/2988358372.py:8: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  df_fg["close_time"] = pd.to_datetime(df_fg["close_time"]).astype(np.int64)


,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,...,DX-Y.NYB_avg,EMA_10,EMA_50,EMA_200,RSI_14,ADX_14,DMP_14,DMN_14,ATR_14,target
0,1517454000000000000,1150.51,1164.99,1125.01,1140.00,7622.31840,1517457599999000000,8.731269e+06,12304,4249.14779,...,324.015003,1130.777,994.6516,866.73115,32.18686,62.004509,13.480052,34.583153,37.751862,1
1,1517457600000000000,1140.00,1154.72,1130.18,1152.51,3899.48396,1517461199999000000,4.461803e+06,8404,1925.63184,...,324.015003,1130.777,994.6516,866.73115,32.18686,62.004509,13.480052,34.583153,37.751862,0
2,1517461200000000000,1152.51,1164.42,1146.50,1159.13,3462.28586,1517464799999000000,4.005314e+06,7597,1814.24364,...,324.015003,1130.777,994.6516,866.73115,32.18686,62.004509,13.480052,34.583153,37.751862,0
3,1517464800000000000,1159.02,1159.02,1140.01,1142.99,3400.69303,1517468399999000000,3.907627e+06,9472,1637.92439,...,324.015003,1130.777,994.6516,866.73115,32.18686,62.004509,13.480052,34.583153,37.751862,1
4,1517468400000000000,1142.99,1151.00,1135.00,1137.10,2589.27888,1517471999999000000,2.962427e+06,6319,1601.73117,...,324.015003,1130.777,994.6516,866.73115,32.18686,62.004509,13.480052,34.583153,37.751862,1


In [14]:
"""
Creating Preproccesing Pipeline:

    Numerical Features (excluding "target") scaled with RobustScaler
    Categorical Feature (only one: "value_classification") encoded with OneHotEncoder

Note: Pending to implement Ordinal Encoder with 5 categories: "Extreme Fear", "Fear", etc

"""
#Select num columns to scale: 

NUM_COL_LIST = df_fg.drop(columns=["target","v"]).select_dtypes(include=["float64","int64"]).columns.values.tolist()
CAT_COL_LIST = ["value_classification"]

# Impute then Scale for numerical variables: 

num_scaler = RobustScaler()

# Encode categorical variables

# cat_encoder = OneHotEncoder()

features = ["Extreme Fear", "Fear", "Neutral","Greed", "Extreme Greed"]

cat_encoder = OrdinalEncoder(
    categories=[features],
    dtype= np.int64,
    handle_unknown="use_encoded_value",
    unknown_value=-1
)



# Paralellize "num_transformer" and "cat_transformer"

preprocessor = ColumnTransformer([
                                ('num_tr', num_scaler, NUM_COL_LIST),
                                ('cat_tr', cat_encoder, CAT_COL_LIST)
                                ],
                                remainder='passthrough')

preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('num_tr', RobustScaler(),
                                 ['open_time', 'open', 'high', 'low', 'close',
                                  'volume', 'close_time', 'quote_asset_volume',
                                  'number_of_trades',
                                  'taker_buy_base_asset_volume',
                                  'taker_buy_quote_asset_volume', 'value',
                                  'btc_avg', '^IXIC_avg', '^GSPC_avg',
                                  'GC=F_avg', '^DJI_avg', '^TNX_avg',
                                  'DX-Y.NYB_avg', 'EMA_10', 'EMA_50', 'EMA_200',
                                  'RSI_14', 'ADX_14', 'DMP_14', 'DMN_14',
                                  'ATR_14']),
                                ('cat_tr',
                                 OrdinalEncoder(categories=[['Extreme Fear',
                                                             'Fear', 'Neutral',
                                                             'Greed',
                                                             'Extreme Greed']],
                                                dtype=<class 'numpy.int64'>,
                                                handle_unknown='use_encoded_value',
                                                unknown_value=-1),
                                 ['value_classification'])])

In [15]:
"""
Fitting and transforming dataframe (df_fg) in preprocessing pipeline
Creating preprocessed dataframe (df_fg_prep)
"""

df_fg_prep = pd.DataFrame(preprocessor.fit_transform(df_fg), columns=preprocessor.get_feature_names_out())
df_fg_prep

,num_tr__open_time,num_tr__open,num_tr__high,num_tr__low,num_tr__close,num_tr__volume,num_tr__close_time,num_tr__quote_asset_volume,num_tr__number_of_trades,num_tr__taker_buy_base_asset_volume,...,num_tr__EMA_50,num_tr__EMA_200,num_tr__RSI_14,num_tr__ADX_14,num_tr__DMP_14,num_tr__DMN_14,num_tr__ATR_14,cat_tr__value_classification,remainder__v,remainder__target
0,-1.002269,0.419480,0.422938,0.410434,0.413702,-0.407670,-1.002269,0.063916,0.090365,-0.373523,...,0.331893,0.256896,-1.160344,2.282005,-0.573847,1.351304,1.245864,1.0,0.875434,1.0
1,-1.002216,0.413700,0.417334,0.413303,0.420582,-0.577712,-1.002216,-0.048730,-0.076502,-0.585617,...,0.331893,0.256896,-1.160344,2.282005,-0.573847,1.351304,1.245864,1.0,0.875434,0.0
2,-1.002163,0.420580,0.422627,0.422363,0.424223,-0.597681,-1.002163,-0.060774,-0.111030,-0.595784,...,0.331893,0.256896,-1.160344,2.282005,-0.573847,1.351304,1.245864,1.0,0.875434,0.0
3,-1.002110,0.424161,0.419680,0.418760,0.415346,-0.600494,-1.002110,-0.063351,-0.030806,-0.611879,...,0.331893,0.256896,-1.160344,2.282005,-0.573847,1.351304,1.245864,1.0,0.875434,1.0
4,-1.002058,0.415344,0.415303,0.415979,0.412107,-0.637556,-1.002058,-0.088289,-0.165711,-0.615183,...,0.331893,0.256896,-1.160344,2.282005,-0.573847,1.351304,1.245864,1.0,0.875434,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37798,0.998259,0.886358,0.885726,0.887274,0.886009,1.556156,0.998259,2.504845,1.431670,1.634326,...,0.811838,0.808994,1.963499,1.494264,1.920203,-1.334257,0.746187,0.0,0.918713,1.0
37799,0.998312,0.886011,0.887609,0.885042,0.882754,1.288277,0.998312,2.191104,1.453876,1.162941,...,0.814445,0.809575,1.764214,1.561814,1.526287,-1.293916,0.778312,0.0,0.918713,1.0
37800,0.998364,0.882750,0.875073,0.883121,0.879305,0.247605,0.998364,0.984909,0.523190,0.145242,...,0.816815,0.810117,1.554915,1.606706,1.357950,-1.232165,0.755115,0.0,0.918713,1.0
37801,0.998417,0.879307,0.878577,0.885986,0.884624,0.098901,0.998417,0.817196,0.511552,0.143731,...,0.819301,0.810706,1.657870,1.656541,1.372269,-1.273762,0.736999,0.0,0.918713,0.0


In [20]:
"""
Defining Subsampling fuction
"""

def subsample_sequence(df, length):
    
    last_possible = df.shape[0] - length - 1
    
    random_start = np.random.randint(0, last_possible)
    X = df[random_start: random_start+length].values
    y = df.iloc[random_start+length+1]['remainder__target']
   
    return X, y


In [21]:
"""
Defining fuction to create X and y using Subsampling function
"""

def get_X_y(df, length_of_observations):
    X, y = [], []
    
    for length in length_of_observations:
        xi, yi = subsample_sequence(df, length)
        X.append(xi)
        y.append(yi)
    
    return X, y

# length_of_observations = np.random.randint(10, 15, 10000)
# X, y = get_X_y(df_fg_prep, length_of_observations)

In [22]:
"""
Splitting data into train and test dataframes
"""

horizon = 1
gap = horizon - 1

len_ = int(0.8*df_fg_prep.shape[0])

df_train = df_fg_prep[:len_]
df_test = df_fg_prep[len_+gap:]

In [23]:
"""
Splitting data into train and test sequences
"""

length_of_observations = np.random.randint(120, 168, 10000)
X_train, y_train = get_X_y(df_train, length_of_observations)

length_of_observations = np.random.randint(120, 168, 1000)
X_test, y_test = get_X_y(df_test, length_of_observations)

In [24]:
"""
Padding train set sequences
"""

X_train_pad = pad_sequences(X_train, dtype='float32', value=-1)

X_train_pad.shape

(10000, 167, 30)

In [25]:
"""
Building RNN Model
"""

normalizer = Normalization()
normalizer.adapt(X_train_pad)
model = Sequential()
model.add(normalizer)
model.add(layers.Masking(mask_value=-1))
model.add(layers.LSTM(10, activation='tanh'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

2022-06-09 16:31:14.963385: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-09 16:31:14.963828: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-09 16:31:14.963915: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-LL2SHNR): /proc/driver/nvidia/version does not exist
2022-06-09 16:31:14.965350: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, None, 30)         61        
 n)                                                              
                                                                 
 masking (Masking)           (None, None, 30)          0         
                                                                 
 lstm (LSTM)                 (None, 10)                1640      
                                                                 
 dense (Dense)               (None, 10)                110       
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 1,822
Trainable params: 1,761
Non-trainable params: 61
_____________________________________________________

In [26]:
"""
Compiling model
"""
model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=0.01), metrics="accuracy")

In [27]:
"""
Fitting model
"""

model.fit(X_train_pad, np.array(y_train), epochs=20, batch_size=64, validation_split=0.3)

Epoch 1/20
110/110 [==============================] - 106s 702ms/step - loss: 0.6954 - accuracy: 0.4933 - val_loss: 0.6927 - val_accuracy: 0.5180
Epoch 2/20
110/110 [==============================] - 68s 621ms/step - loss: 0.6939 - accuracy: 0.5037 - val_loss: 0.6947 - val_accuracy: 0.4920
Epoch 3/20
110/110 [==============================] - 69s 624ms/step - loss: 0.6937 - accuracy: 0.5033 - val_loss: 0.6922 - val_accuracy: 0.5113
Epoch 4/20
110/110 [==============================] - 65s 595ms/step - loss: 0.6931 - accuracy: 0.5089 - val_loss: 0.6920 - val_accuracy: 0.5177
Epoch 5/20
110/110 [==============================] - 63s 577ms/step - loss: 0.6922 - accuracy: 0.5204 - val_loss: 0.6916 - val_accuracy: 0.5340
Epoch 6/20
110/110 [==============================] - 68s 617ms/step - loss: 0.6911 - accuracy: 0.5290 - val_loss: 0.6931 - val_accuracy: 0.5120
Epoch 7/20
110/110 [==============================] - 62s 564ms/step - loss: 0.6904 - accuracy: 0.5214 - val_loss: 0.6921 - val_a

KeyboardInterrupt: 